## 청와대 국민제안 크롤링

In [13]:
import pandas as pd
# load contents and data from website
import requests
# parsing for requst tag 
from bs4 import BeautifulSoup as bs
import random
import time
# display data on process
from tqdm import tqdm, trange
import re

In [29]:
suggestion_list = []

def get_suggestion_list(pnum):
    # print(f'year_month:{year_month}, pnum: {pnum})
    base_url = f"https://www.innogov.go.kr/ucms/ogp/sug/list.do?pnum={pnum}&menuNo=300011&orderKey=registDtDesc"
    
    # request base_url
    response = requests.get(base_url)
    
    if response.status_code == 200:
        # get html tag by beautifulSoup
        html = bs(response.text, 'html.parser')
        
        # html 태그중 selector를 통해 목록에 있는 다른 게시글을 가져오도록 한다.
        tags = html.select('#content > div.suggestion_list > ul')[0].find_all('a')
        
        if not tags:
            return (suggestion_list)
        else:
            # tags 에서 다른 게시글을 가져와 suggetion_list에 담아준다.
            for tag in tags:
                suggestion_list.append(tag['href'])
        
        pnum += 1
        
        # 페이지 번호를 하나씩 더해서 재귀호출을 한다.
        get_suggestion_list(pnum)
        print(pnum)
        
    else:
        return(suggestion_list)


In [30]:
pnum = 1

get_suggestion_list(pnum)
suggestion_list

['/ucms/ogp/sug/view.do?menuNo=300011&sgId=280&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=279&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=278&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=277&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=276&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=275&pnum=2',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=274&pnum=2',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=272&pnum=2',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=271&pnum=2',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=270&pnum=2',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=269&pnum=3',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=268&pnum=3',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=267&pnum=3',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=266&pnum=3',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=265&pnum=3',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=264&pnum=4',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=263&pnum=4',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=262&p

In [31]:
print(len(suggestion_list))
suggestion_list[:3]

236


['/ucms/ogp/sug/view.do?menuNo=300011&sgId=280&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=279&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=278&pnum=1']

## parse contents from board

In [ ]:
# parsing pre-view for id value
# u.split('start str')[start str place].split('end str')['end']
u = 'view.do?menuNo=300011&sgId=280&pnum=1'
u.split('=')[2].split('&')[0]

In [49]:
# url = '/ucms/ogp/sug/view.do?menuNo=300011&sgId=280&pnum=1'
# return url of suggestion list and crawling contents
def get_suggestion_content(url):
    
    article = []
    base_url = f"https://www.innogov.go.kr{url}"
    response = requests.get(base_url)

    if response.status_code == 200:

        html = bs(response.text, 'html.parser')

        title = html.select(
            '#content > div.vveiw_box1 > dl > dt')[0].get_text(strip=True)

        ## parse category, start, end, author from description
        desc = html.select(
            '#content > div.vveiw_box1 > div.vveiw_name > ul > li > dl > dd')    

        category = desc[0].get_text(strip=True)
        start = desc[1].get_text(strip=True)
        end = desc[2].get_text(strip=True)
        author = desc[3].get_text(strip=True)

        # parse contents
        content = html.select(
            '#content > div.vveiw_box1 > div.vveiw_cont > div > pre')[0].get_text(strip=True)

        # parse the number of vote
        vote = html.select('#counter')[0].get_text(strip=True)

        sgId = url.split('=')[2].split('&')[0]
        
        article.append(sgId)
        article.append(title)
        article.append(category)
        article.append(content)
        article.append(start)
        article.append(end)
        article.append(vote)
        article.append(author)
        
        time.sleep(random.randint(1,2))
        
        return article

In [50]:
data = []
for i, suggestion in tqdm(enumerate(suggestion_list)):
    s = get_suggestion_content(suggestion)
    data.append(s)

236it [08:12,  2.09s/it]


In [52]:
column_names = ['sgId', 'title', 'category', 'content', 
                'start', 'end', 'vote', 'author']
column_names

['sgId', 'title', 'category', 'content', 'start', 'end', 'vote', 'author']

In [53]:
data = pd.DataFrame(data, columns=column_names)
data.head()

,sgId,title,category,content,start,end,vote,author
0,280,행정기관 소속 위원회 위원 개인정보 및 안건 공개 기준에 관한 법적 근거 마련,일반행정,"ㅁ 현황ㅇ 법령에 따라 설치되는 행정기관 소속 위원회들이 다수 운영되고 있으며, ...",2019.10.16,2020.01.13,0,법제혁신
1,279,초과근무 줄이기!,일반행정,1. 문제점* 초과근무를 많이 할수록 일을 열심히 하고 있다고 생각하는 조직 문화*...,2019.10.16,2020.01.13,2,ㅁㄴㅁ
2,278,원스톱 의원요구자료 제출 시스템 구축,일반행정,​​□ 제안제목○ 원스톱 의원요구자료 제출 시스템 구축□현황 및 문제점○ 각부처 별...,2019.10.15,2020.01.12,0,연시랑
3,277,예산을 공평하게 분배,사회통합,제목 : 예산을 부처별 공평하게 분배후 차후평가문제점 : 1. 예산편성 삭감원칙으...,2019.10.15,2020.01.12,3,슈퍼맨119
4,276,공무원 인트라넷 통합메신저,일반행정,"부처별,기관별로 자체 인트라넷 메신저가 활성화되며,신속한 업무 추진과 협조에 큰 도...",2019.10.15,2020.01.12,0,페이소스


In [54]:
data['category'].value_counts()

일반행정     41
안전       33
보건복지     32
기타       31
인권/평등    21
국민참여     17
상생협력     15
환경       14
사회통합     13
공동체      10
반부패청렴     9
Name: category, dtype: int64

In [55]:
data.to_csv('./suggestion.csv', index=False)